In [1]:
from pymongo import MongoClient

mongodb是一个轻量级的数据库

数据格式为JSON风格

## 连接到mongodb数据库 

In [2]:
#默认连接本地数据库
client = MongoClient()    

## 创建数据库

In [3]:
db = client['test_database']

### 查看数据库
> client.database_names()

###  删除数据库
> client.drop_database()

## 从数据库中选择集合

In [4]:
collection = db['test_collection']

### 查看下面的集合
> db.collection_names()

## 操作符

### 第一部分
> $set：替换某个field的值     

> $unset：取消某个field       

> $addToSet：插入值至array

> $inc：+1

> $mul：乘

> $rename：重命名     		 

> $min：指定值比存在值小则更新

> $max                 

### 第二部分

> \$lt：小于
>> {'age': {'$lt': 20}}

> \$gt：大于
>> {'age': {'$gt': 20}}

> \$lte：小于等于
>> {'age': {'$lte': 20}}

> \$gte：大于等于
>> {'age': {'$gte': 20}}

> \$ne：不等于
>> {'age': {'$ne': 20}}

> \$in：在范围内
>> {'age': {'$in': [20, 23]}}

> \$nin：不在范围内
>> {'age': {'$nin': [20, 23]}}

### 第三部分
> \$regex匹配正则
>> {"name": {"$regex": "^M.*"}}

> \$exists属性是否存在
>> {"name": {"$exists": True}}   
>> #判断name属性存在

> \$type类型判断
>> {"age": {"$type": "int"}}
>> #判断age的类型为int

> \$mod数字模操作
>> {"age": {"$mod": [5, 0]}}
>> #判断年龄模5余0

> \$text文本查询
>> {"$text": {"$search": "Mike"}}

> \$where高级条件查询
>> {"$where": "obj.fans_count == obj.follows_count"} 

## 插入数据
> 插入单条数据：insert_one(document)
>> collection.insert_one({“x”:1})

> 插入多条数据：insert_many(documents)
>> collection.insert_many([{“x”:1} for I in range(2)])

## 查询数据
> 查询单条数据：find_one()
>> collection.find_one({'主题页面': href})

> 查询多条数据：find()
>> collection.find({'age': {'$gt': 20}})

>> collection.find() - 查看所有数据 

> 查询并操作，返回对应内容
>> find_one_and_delete(filter, projection=None, sort=None)
>>> collection.find_one_and_delete({"x":1})

>> find_one_and_replace(filter, replacement, projection=None, sort=None)
>>> collection. find_one_and_replace({"x": 1}, {"y": 1})

>> find_one_and_update(filter, update, projection=None, sort=None)
>>> collection. find_one_and_update({"_id": 665}, {"\\$inc": {"count": 1}, '$set': {"done": True}})

无论是否查询到，find()返回值均不为F

## 替换数据
> replace_one(filter, replacement, upsert=False) 
>> collection.replace_one({“x”:1},{“y”:1})
>> #upsert=True ：若匹配文档不存在，则插入新的

## 删除数据
> 删除单条数据
>> delete_one()

> 删除多条数据
>> delete_many()

> 删除集合
>> collection.drop()

> 删除集合里所有记录
>> collection.remove()

## 更新数据
> 更新单条数据：update_one(filter, update, upsert=False)
>> collection.update_one({'x': 1}, {'$inc': {'x': 3}})

> 更新多条数据：update_many(filter, update, upsert=False)
>> collection.update_many({'x': 1}, {'$inc': {'x': 3}})
>> #可以通过collection.update_many({}, {condition})跳过筛选条件

## 其它
> 计数：count()
>> collection.find({"author": "Mike"}).count()

> 排序：sort()
> #1为升序排列, -1为降序排列；或者使用pymongo.ASCENDING
>> collection.find().sort({"likes":-1})

>>collection.find().sort("likes”, pymongo.ASCENDING)

> 偏倚
>> skip()：指定忽略的个数
>>> collection.find().sort('name', pymongo.ASCENDING).skip(2)
>>> #skip(2)为忽略前2个元素，得到第三个及以后的元素

>> limit()：指定选取的个数
>>> collection.find().sort('name', pymongo.ASCENDING).skip(2).limit(2)

> 索引：createIndex()
> #能够极大的提高查询的效率,升序降序同sort（）
>> collection.createIndex({“title”:1})                
>> #单索引

>> collection.createIndex({“title”:1,“description”:-1})    	
>> #多索引

> 寻找key的特异值
>> distinct(key, filter=None, **kwargs)

## 对数组进行操作
> \$all：判断数组属性是否包含全部条件
>> collection.find({"data":{"\\$all":[2,3,4]}})

> \$size：匹配数组属性元素数量
>> collection.find({"data":{"\\$size":3}})

> \$type：判断属性
>> collection.find({"t":{"\\$type":1}})
>> #其中1：数字，2：字符串，4：array，8：boolean

> \$each：添加多个元素
>> collection.update_many({"name":"user2",{"\\$addToSet":{"data":{"\\$each":[1,2,3,4]}}}})

> \\$push 和 \$pushAll：向数组添加元素
> #而$addToSet仅在该元素不存在时才添加

> \$pop：移除数组属性的元素(数组下标)
>> collection.update_many({"name":"user2",{"\\$pop":{"data":1}})
>> #其中1：移除最后一个元素; -1：移除第一个元素

> \$pull：按值移除
>> collection.update_many({"name":"user2",{"\\$pull":{"data":2}})

> \$pullAll：移除所有符合提交的元素
>> collection.update_many({"name":"user2",{"\\$pullAll":{"data":[3,5,6]}})

## 示例 

StockConceptMongo()运用以下功能：
>  "$pull":{concept_type:concept}：剔除相关概念

>  concept_type:{"\$exists": True}：判断属性是否存在concept_type

>  concept_type:{"\$all":[concept]}：判断concept_type中是否包含特定概念

> \\$set, \$addToSet

In [ ]:
class StockConceptMongo():

    def __init__(self,timeout=300):
        self.client = MongoClient()
        self.db = self.client["stock"]
        self.collection = self.db["stockConcept"]
        self.timeout = timeout

    def find_one(self,stock_code):
        return self.collection.find_one({"stock_code":stock_code})

    def pull(self,stock_code,concept_type,concept):
        self.collection.update({"stock_code":stock_code},{"$pull":{concept_type:concept}})

    def update(self,stock_code,stock_name,concept_type,concept):
        #2018-11-12 添加若股票出现新概念，则提醒 -- 还需要添加新概念提醒
        if not self.collection.find_one({"stock_code": stock_code, concept_type: {"$exists": True}}) or \   
                not self.collection.find_one({"stock_code":stock_code,concept_type:{"$all":[concept]}}):    
            print("%s出现新概念%s" %(stock_code,concept))
            self.collection.update({"stock_code":stock_code},
                                   {"$set":{"stock_name":stock_name},"$addToSet":{concept_type:concept}},
                                    upsert=True)
        else:
            pass

YesterdayZtMongo运用以下功能：
> "\$inc":{"con_zt_days":1}：+1

> delete_many({"today": {"\$ne": self.today}})：删除昨日Zt数据

> update_many({"jqkConcept":{"\$all":["昨日涨停"]}},{"\$pull":{"jqkConcept":"昨日涨停"}})：删除昨日Zt概念

> sort("con_zt_days",pymongo.DESCENDING)[0]["con_zt_days"]：排序

In [ ]:
class YesterdayZtMongo():
    """
    先利用昨日涨停股计算该该概念的今日涨幅
    再更新今日涨停股
    最后删除昨日涨停股
    """

    def __init__(self,timeout=300):
        self.client = MongoClient()
        self.db = self.client["stock"]
        self.collection = self.db["yesterdayZt"]
        self.concept_collection = self.db["stockConcept"]
        self.timeout = timeout
        self.today = judge_date()

    def find_one(self,stock_code):
        return self.collection.find_one({"stock_code":stock_code})

    def delete_one(self,stock_code):
        self.collection.delete_one({"stock_code":stock_code})

    def get_today_ave_pchange(self,df):
        stockList= []
        for item in self.collection.find():
            stockList.append(item["stock_code"])
        filter = df.ix[stockList]
        aveChange = round(filter["changepercent"].sum()/len(stockList),2)
        print("昨日涨停的今日涨幅: %s" % (aveChange))
        return aveChange

    def update(self,stock_code,stock_name):
        if self.db["newStock"].find_one({"stock_code":stock_code,"kb":False}):
            pass
        else:
            if self.collection.find_one({"stock_code":stock_code}):
                if self.collection.find_one({"stock_code":stock_code})["today"] != self.today:
                    self.collection.update({"stock_code": stock_code},
                                            {"$set":{"stock_name":stock_name,"today":self.today},"$inc":{"con_zt_days":1}},
                                            upsert=True)
                con_zt_days = self.collection.find_one({"stock_code":stock_code})["con_zt_days"]
                print("%s %s连续%s天涨停" %(stock_code,stock_name,con_zt_days))
            else:
                con_zt_days = max(get_conZt_num(stock_code),1)
                self.collection.insert_one({"stock_code":stock_code,"stock_name":stock_name,"today":self.today,"con_zt_days":con_zt_days})

    def refresh(self):
        self.collection.delete_many({"today": {"$ne": self.today}})

    def trasnfer2concept(self):
        self.concept_collection.update_many({"jqkConcept":{"$all":["昨日涨停"]}},{"$pull":{"jqkConcept":"昨日涨停"}})
        for item in self.collection.find({"today":self.today}):
            stock_code = item["stock_code"]
            stock_name = item["stock_name"]
            self.concept_collection.update({"stock_code":stock_code},
                                          {"$set":{"stock_name":stock_name},"$addToSet":{"jqkConcept":"昨日涨停"}},
                                           upsert=True)
    def get_max_zt_days(self):
        return self.collection.find().sort("con_zt_days",pymongo.DESCENDING)[0]["con_zt_days"]